## α-β-Pruning

In [ ]:
from IPython.core.display import HTML
with open("style.html", "r") as file:
    css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-artificial-intelligence.ipynb
%run ./nmm-heuristic.ipynb
%run ./nmm-symmetry.ipynb
import time

Für die Implementierung des *α-β-Pruning* Algorithmus wird eine Klasse `AlphaBetaPruning` implementiert, die ebenfalls von der zuvor definierten Klasse `ArtificialIntelligence` erbt. Auch hier wird die Funktion `bestMoves` überschrieben, die später genauer definiert wird, und den Konstruktor `__init__` implementiert. Der Konstruktor besitzt drei optionale Parameter, die den Algorithmus konfigurieren können:

* die `max_states` Einstellung setzt die maximale Anzahl an Zusänden, die betrachtet werden sollen;
* die `symmetry` Einstellung legt fest, ob alle symmetrischen Spielfelder zu einem Zustand berechnet werden sollen und dann ebenfalls in der Transpositionstabelle abgelegt werden sollen;
* durch den `weights` Parameter können die Gewichtungen der zuvor definierten Heuristik bestimmt werden.

Zusätzlich initialisiert der Konstruktor das Attribute `cache` mit einem leeren `dict`. Dieses wird später als Transpositionstabelle, bzw. als Sortierungsgrundlage in der Funktion `orderMoves` für *Iterative Deepening* verwendet.

Der Standardwert für das `weights` Attribut muss explizit im Funktionsrumpf gesetzt werden, damit Probleme bei der späteren Verwendung des `multiprocessing` Paketes verhindert werden können.

1. Wenn der Standardwert bereits in der Funktionssignatur gesetzt wird, wird dieser Wert beim ersten Laden der `AlphaBetaPruning` Klasse gespeichert. Dieser Wert beinhaltet dann eine Instanz der unmittelbar zuvor geladenen `HeuristicWeights` Klasse.
2. Läde man darauf hin das `Minimax` Notebook, dann lädt dieses die `HeuristicWeights` Klasse erneut. Dabei wird jedoch eine neue Klasse unter dem gleichen Namen, aber an einem anderen Ort im Arbeitsspeicher angelegt.
3. Wird nun ebenso der Standardwert des `weights` Parameters im `Minimax` Algorithmus beim ersten Laden gesetzt, dann teilen sich zwar beide Standardwerte die gleiche Implementierung der `HeuristikWeights` Klasse, zeigen jedoch auf verschiedene Addressen im Arbeitsspeicher.
4. Dies wird zum Problem, wenn die Objekte mit Hilfe des `pickel` Paketes serialisiert werden, damit sie im `multiprocessing` Paket über die Threadgrenzen gesendet werden können. Hier wird ein Fehler ausgelöst, wenn der Name der Klassen übereinstimmt, jedoch auf verschiedene Addressen im Speicher verwiesen wird.

In [ ]:
class AlphaBetaPruning(ArtificialIntelligence):
    def __init__(self, max_states=25_000, symmetry=True, weights=None):
        self.cache = {}
        
        self.max_states = max_states
        self.symmetry = symmetry
        self.weights = weights
        if self.weights is None:
            self.weights = HeuristicWeights()
    
    def bestMoves(self, state, player):
        pass

Für Debuggingzwecke wird eine `__repr__` Funktion implementiert, die eine String-Repräsentation aus einer `AlphaBetaPruning` Instanz mit allen Einstelungen erstellt.

In [ ]:
def __repr__(self: AlphaBetaPruning):
    return f"AlphaBetaPruning(max_states={self.max_states}, symmetry={self.symmetry}, weights={self.weights})"

AlphaBetaPruning.__repr__ = __repr__
del __repr__

### Iterative Tiefensuche

Die Iterative Tiefensuche bietet die Möglichkeit eine konstante Anzahl an Zuständen betrachten zu können.
Dieses bietet den Vorteil gegenüber fest definierten Tiefenlimits, dass bei weniger komplexen Bäumen die Suche weiter fortgesetzt werden kann und die Antwortzeit an den Nutzer durch die gleiche Anzahl an Zuständen nahezu konstant bleibt.
Dies sorgt gerade in Phase 2 des Mühle-Spiels dafür, dass die Spielzüge um einiges weiter im Voraus berechnet werden können.
Andererseits verhindert es jedoch auch bei sehr komplexes Suchbäumen eine verlängerte Antwortzeit.

In der Umsetzung bedeutet das, dass die maximale Rekursionstiefe bei einem Aufruf der `bestMoves()`-Funktion zunächst immer bei `limit=1` liegt. Wenn nach der Betrachung aller Zustände mit Verwendung der Rekursionstiefe noch keine `IterativeMaxCountException()` geworfen wurde, wird das Maximum um eins erhöht.

Die gewünschte maximale Anzahl an Zuständen `maxCount` kann im `AlphaBetaPruning`-Konstruktor mit Hilfe der Option `max_states` festgelegt werden.

#### Exception für das Erreichen der maximalen Zuständen `IterativeMaxCountException`

Die Exception `IterativeMaxCountException` wird aufgerufen, wenn ein Fehler aufgrund des Erreichens der maximal zu betrachtenden Zustände bei der iterativen Tiefensuche erzeugt werden soll.
Der Parameter `maxCount` im Konstruktor der Fehlermeldung nimmt dabei die erreichte maximale Anzahl an besuchten Zuständen an, um diese in der Fehlermeldung wiedergeben zu können.

In [ ]:
class IterativeMaxCountException(Exception):
    def __init__(self, maxCount):
        super().__init__(f"Reached max count ({maxCount}) of iterative deepeing")    

#### Hilfsfunktion zum Überprüfen der maximal zu besuchenden Zustände `checkMaxStates`

Die Hilfsfunktion `checkMaxStates` zählt die errechneten und besuchten Zustände für die Implementierung der *iterativen Tiefensuche*. Ist die Anzahl der maximal erlaubten Zustände überschritten, wird ein Fehler ausgelöst, damit der Algorithmus abbricht und ein Ergebnis zurück gegeben werden kann. Diese Ausnahmebetrachtung ist in der Funktion `bestMoves` implementiert.

In [ ]:
def checkMaxStates(self):
    self.visited += 1
    if (self.visited >= self.max_states):
        raise IterativeMaxCountException(self.max_states)

AlphaBetaPruning.checkMaxStates = checkMaxStates
del checkMaxStates

#### Sortieren der Zustände

Die Funktion `orderMoves` sortiert eine Liste von Zuständen `states` für einen Spieler
`player` entsprechend der für diese Zustände erreichten Werte aus der vorherigen Iteration
der *Iterativen Tiefensuche*. So soll erreicht werden, dass vielversprechende
Zustände früher betrachtet werden und sich somit Teilbäume eventuell früher abschneiden lassen.

In [ ]:
def orderMoves(self, states, player, limit):
    return sorted(
        states,
        key     = lambda state: self.cache.get((state, player, limit-1), (0, -1, 1))[0],
        reverse = True
    )

AlphaBetaPruning.orderMoves = orderMoves
del orderMoves

### Implementierung *α-β-Pruning*
#### Hilfsfunktion zum Füllen der Transpositionstabelle `writeCache`

Die Hilfsfunktion `writeCache` legt den gegebenen Zustand `state` und Spieler `player` mit samt der errechneten Werte `value`, `alpha` und `beta` in der Transpositionstabelle (`cache`) ab. Ist zusätzlich die Einstellung `symmetry` aktiviert, werden auch die errechneten Spielfelder in der Transpositionstabelle gespeichert.

In [ ]:
def writeCache(self, state, player, value, alpha, beta, limit):
    if self.symmetry:
        for symmetricState in findSymmetries(state):
            self.cache[(symmetricState, player, limit)] = (value, alpha, beta)
    else:
        self.cache[(state, player, limit)] = (value, alpha, beta)

AlphaBetaPruning.writeCache = writeCache
del writeCache

#### Hilfsfunktion zum Abgleich mit der Transpositionstabelle `value`

Die `value` Funktion ist ein Wrapper für die eigentliche Implementierung des *α-β-Pruning*
in der Funktion `alphaBeta`. Dieser Wrapper überprüft die Transpositionstabelle *cache* auf das Vorhandensein
eines bereits berechneten Wertes für eine Kombination aus `state`, `player`, und `limit`.
Ist ein Wert vorhanden, wird dieser auf seine Validität überprüft.
Wenn diese Überprüfung fehlschlägt, oder kein Wert vorhanden ist, wird dieser mithilfe der `alphaBeta`-Funktion berechnet
und in der Transpositionstabelle *cache* abgespeichert.

Ein Ergebnis aus dem Cache ist valide, solange das Intervall `alpha` und `beta` aus den Parametern innerhalb des im Cache verwendeten Intervalls `a` und `b` liegt. Also das Intervall des Caches muss genereller sein, als das Intervall aus den Parametern.

Für Auswertungszwecke wird außerdem bei jedem der drei möglichen Fälle ein Zähler erhöht.

In [ ]:
def value(self, state, player, alpha, beta, limit):
    if (state, player, limit) in self.cache:
        (val, a, b) = self.cache[(state, player, limit)]
        if a <= alpha and beta <= b:
            self.cache_hit += 1
            return val
        else:
            alpha = min(alpha, a)
            beta  = max(beta , b)
            val   = self.alphaBeta(state, player, alpha, beta, limit)
            self.writeCache(state, player, val, alpha, beta, limit)
            self.cache_invalid += 1
            return val
    else:
        val = self.alphaBeta(state, player, alpha, beta, limit)
        self.writeCache(state, player, val, alpha, beta, limit)
        self.cache_miss += 1
        return val

AlphaBetaPruning.value = value
del value

#### Funktion zur Berechnung des Wertes eines Spielzustands `alphaBeta`

Die Funktion `alphaBeta` beinhaltet nun die eigentliche Implementierung des *α-β-Pruning*.

* Wie zuvor beim *Minimax-Algorithmus*, wird der `utility` Wert zurückgegeben, falls das Spiel in dem State `s` beendet (`finished`) ist.
* Ebenfalls äquivalent wird der `heuristic` Wert verwendet, sobald das Rekursionslimit (`limit`) erreicht wird.
* Zusätzlich wird mithilfe der Funktion `checkMaxStates` überprüft, ob die maximal zu betrachtende Anzahl an Zuständen erreicht wurde.
* Der eigentliche α-β-Pruning Alogrithmus errechnet rekursiv mit Hilfe des Caches (`value`) den Wert eines Zuges. Hierbei wird der erste Wert der nächsten States verwendet, der größer oder gleich der oberen Grenze `beta` ist.

In [ ]:
def alphaBeta(self, state, player, alpha, beta, limit):
    self.checkMaxStates()
    
    if limit == 0:
        return heuristic(state, player, self.weights)

    states = nextStates(state, player)
    if finished(state, player, ns=states):
        return utility(state, player, ns=states)

    val = alpha
    for ns in self.orderMoves(states, player, limit):
        val = max(val, -self.value(ns, opponent(player), -beta, -alpha, limit-1))
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

AlphaBetaPruning.alphaBeta = alphaBeta
del alphaBeta

#### Funktion zur Auswahl des bestmöglichen Zuges `bestMoves`

Die Funktion `bestMoves` berechnet rekursiv den geschätzten Wert aller möglichen Züge für einen Spieler `player` und wählt die Züge mit dem besten Wert aus. Diese rekursive Suche wird solange ausgeführt, bis die maximale Anzahl der zu betrachtenden Zustände erreicht ist.
Erreicht wird dies durch das Abfangen einer Exception, welche beim Erreichen dieser Grenze ausgelöst wird und somit die Endlosschleife beendet.
Wenn für eine Rekursionstiefe alle Zustände betrachtet wurden, wird diese automatisch erhöht.
Abschließend werden aus den erhaltenen Zuständen die Zustände mit dem besten Wert ausgewählt und zurückgegeben.

Zusätzlich werden Informationen gesammelt, die die genaueren Abläufe im Algorithmus abbilden:
* `runtime` ist die Rechendauer der Funktion in Sekunden;
* `limit` ist die erreichte maximale Rekursionstiefe;
* `visited` ist die Anzahl der besuchten Zustände;
* `max_states` ist die Anzahl der maximal zu besuchenden Zustände;
* `cache_hit` ist die Anzahl der Berechenungen, die durch die Tanspositionstabelle (`cache`) eingespart werden konnten;
* `cache_invalid` ist die Anzahl der Berechnungen, die durchgeführt werden mussten, da der in der Transpositionstabelle vorhandene Wert nicht zu verwenden war;
* `cache_miss` ist die Anzahl der Berechnungen, die trotz der Transpositionstabelle durchgeführt werden mussten, da kein Eintrag gefunden wurde.

In [ ]:
def bestMoves(self, state, player):
    # Start clock
    start = time.time()
    
    # Reset counter
    self.visited = 0
    self.cache_hit = 0
    self.cache_invalid = 0
    self.cache_miss = 0
    
    states = nextStates(state, player)
    moves = [(0, s) for s in states]

    limit = 1
    while True:
        try:
            moves = [
                (-self.value(s, opponent(player), -1, 1, limit), s)
                for s in states
            ]
            limit += 1
        except IterativeMaxCountException:
            break
    
    maximum = max(v for (v, s) in moves)
    bestMoves = [s for (v, s) in moves if v == maximum]
    
    end = time.time()
    return BestMoves(
        bestMoves,
        maximum,
        # Collect debug information
        {   "runtime": end - start,
            "limit": limit,
            "visited": self.visited,
            "max_states": self.max_states,
            "cache_hit": self.cache_hit,
            "cache_invalid": self.cache_invalid,
            "cache_miss": self.cache_miss,   }
    )

AlphaBetaPruning.bestMoves = bestMoves
del bestMoves